In [2]:
import os
import shutil
import random

# Set the paths to your dataset and the train/test folders
data_dir = "/kaggle/input/covid-19/Covid19"
train_dir = "/kaggle/working/train"
test_dir = "/kaggle/working/test"

# Create the train and test folders if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Create subfolders for the classes (Normal and Covid-19)
os.makedirs(os.path.join(train_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(train_dir, "COVID"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "Normal"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "COVID"), exist_ok=True)

# Set the train/test split ratio (e.g., 0.8 for 80% train, 0.2 for 20% test)
train_ratio = 0.8

# Loop through the classes
for class_name in ["Normal", "COVID"]:
    class_dir = os.path.join(data_dir, class_name)
    files = os.listdir(class_dir)
    
    # Shuffle the files randomly
    random.shuffle(files)
    
    # Calculate the number of files for train and test sets
    num_train = int(len(files) * train_ratio)
    
    # Copy the files to the train and test folders
    for i, file in enumerate(files):
        src = os.path.join(class_dir, file)
        if i < num_train:
            dst = os.path.join(train_dir, class_name, file)
        else:
            dst = os.path.join(test_dir, class_name, file)
        shutil.copy(src, dst)

In [1]:
%pip install -q flwr[simulation] 


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.3 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf 23.8.0 requires cuda

In [6]:
import numpy as np
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import flwr as fl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import IPython.display as display_output
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from collections import OrderedDict
from typing import List, Tuple
from IPython.display import display, clear_output
from IPython.display import display, HTML, clear_output
from flwr.common import Metrics
from flwr.server.strategy import FedAdagrad

global num_c, trainloaders, valloaders, testloader, model, classes
num_clients = 15

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Required for Kaggle to work with PyTorch

torch.cuda.is_available()  # Check if GPU is available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE} using PyTorch {torch.__version__}")

# Dataset paths
dataset_paths = '/kaggle/working/'

datatrain = f'{dataset_paths}/train'
datatest = f'{dataset_paths}/test'

BATCH_SIZE = 32
img_size = (224, 224)

# Download and transform
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = ImageFolder(datatrain, transform=transform)
testset = ImageFolder(datatest, transform=transform)
size_label = widgets.Label()
size_label.value = f'The size of data: {len(trainset) + len(testset)}'
print(size_label)

# Get the classes from the trainset
classes = trainset.classes
num_c = len(classes)

# Split training set into num_clients partitions to simulate the individual dataset
partition_size = len(trainset) // num_clients
remainder = len(trainset) % num_clients
lengths = [partition_size + remainder] + [partition_size] * (num_clients - 1)
datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

# Split each partition into train/val and create DataLoader
trainloaders = []
valloaders = []
for ds in datasets:
    len_val = len(ds) // 10  # 10 % validation set
    len_train = len(ds) - len_val
    lengths = [len_train, len_val]
    ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
    trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
    print("hi")
    valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)
    
class Net(nn.Module):
    def __init__(self, num_c):
        super(Net, self).__init__()
        self = self.to(DEVICE)
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 264)
        self.fc2 = nn.Linear(264,num_c)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.pool3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x

def train(net, trainloader, valloader, epochs: int, patience: int = 5, verbose=True, lr=0.001):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)  # Use a smaller learning rate
    net.train()
    best_val_acc = 0.0
    early_stop_counter = 0
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        val_loss, val_acc = test(net, valloader)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping after {epoch + 1} epochs.")
                break

        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}, val loss {val_loss}, val accuracy {val_acc}")

def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

 
def aggregate_FedAdagrad(metrics: List[Tuple[int, dict]]) -> dict:
    total_examples = sum([num_examples for num_examples, _ in metrics])
    weighted_sum = {key: np.zeros_like(metrics[0][1][key]) for key in metrics[0][1]}  # Initialize weighted sum with zeros

    for num_examples, metric in metrics:
        for key in metric:
            # Multiply each metric value by the number of examples and add to the weighted sum
            weighted_sum[key] += num_examples * metric[key]

    # Divide the weighted sum by the total number of examples to compute the weighted average
    weighted_avg = {key: weighted_sum[key] / total_examples for key in weighted_sum}

    return weighted_avg

def fit_metrics_aggregation_fn(fit_metrics):
    # Aggregate and return custom metric (e.g., weighted average)
    accuracies = [num_examples * m["accuracy"] for num_examples, m in fit_metrics]
    examples = [num_examples for num_examples, _ in fit_metrics]
    return {"accuracy": sum(accuracies) / sum(examples)}

class FlowerNumPyClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)
    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader,self.valloader, epochs=1)
        
        # Get the accuracy on the training set
        correct, total = 0, 0
        self.net.eval()
        with torch.no_grad():
            for images, labels in self.trainloader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        
        return get_parameters(self.net), len(self.trainloader), {"accuracy": float(accuracy)}
    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}
    
    
def client_fn(cid) -> FlowerNumPyClient:
    net = Net(num_c).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    flower_numpy_client = FlowerNumPyClient(cid, net, trainloader, valloader)
    return flower_numpy_client.to_client()

def run_simulation(num_round, strategy_dropdown):
    if strategy_dropdown == "FedAdagrad":
        # Start simulation with FedAdagrad strategy

        # Create FedAdagrad strategy
        strategy = FedAdagrad(
            fraction_fit=0.8,  # Increase the fraction of clients participating in training
            fraction_evaluate=0.5,
            min_fit_clients=num_clients - 1,
            min_evaluate_clients=num_clients - 2,
            min_available_clients=num_clients,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(num_c))),
            evaluate_metrics_aggregation_fn=aggregate_FedAdagrad,
            fit_metrics_aggregation_fn=fit_metrics_aggregation_fn,
        )

    # Update with your specific simulation code
    fl.common.logger.configure(identifier="myFlowerExperiment", filename="log.txt")
    output = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=num_round),
        strategy=strategy,
        client_resources={"num_cpus": 1, "num_gpus": 1},
    )

run_simulation(15, "FedAdagrad")

Training on cuda using PyTorch 2.1.2
Label(value='The size of data: 13808')
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


INFO :      Starting Flower simulation, config: num_rounds=15, no round_timeout
2024-05-07 13:45:17,463	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'memory': 17407959860.0, 'GPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.19.2.2': 1.0, 'object_store_memory': 8703979929.0, 'CPU': 4.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 14 clients (out of 15)
(pid=316) 2024-05-07 13:45:21.926086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN fa

(ClientAppActor pid=316) [Client 8] fit, config: {}
(ClientAppActor pid=316) Epoch 1: train loss 0.0243433378636837, accuracy 0.755656108597285, val loss 0.01867356937225551, val accuracy 0.8082191780821918
(ClientAppActor pid=315) [Client 0] fit, config: {}
(ClientAppActor pid=315) [Client 10] fit, config: {}
(ClientAppActor pid=315) Epoch 1: train loss 0.022912852466106415, accuracy 0.6711915535444947, val loss 0.023816780276494483, val accuracy 0.7534246575342466 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(ClientAppActor pid=315) [Client 2] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=315) Epoch 1: train loss 0.025063401088118553, accuracy 0.6666666666666666, val loss 0.020008179014676237, val accuracy 0.7397260273972602 [repeated 2x across cluster]
(ClientAppActor pid=315) [Client 12]

INFO :      aggregate_fit: received 14 results and 0 failures
INFO :      configure_evaluate: strategy sampled 13 clients (out of 15)


(ClientAppActor pid=316) [Client 7] evaluate, config: {}
(ClientAppActor pid=316) [Client 4] fit, config: {}
(ClientAppActor pid=316) Epoch 1: train loss 0.018474269658327103, accuracy 0.7541478129713424, val loss 0.018579798201992088, val accuracy 0.7397260273972602
(ClientAppActor pid=316) [Client 1] evaluate, config: {} [repeated 12x across cluster]


...

INFO :      aggregate_evaluate: received 13 results and 0 failures
INFO :      
INFO :      [ROUND 15]
INFO :      configure_fit: strategy sampled 14 clients (out of 15)


(ClientAppActor pid=315) [Client 9] fit, config: {}
(ClientAppActor pid=315) Epoch 1: train loss 0.01712004281580448, accuracy 0.7677224736048266, val loss 0.02222272467939821, val accuracy 0.7671232876712328
(ClientAppActor pid=315) [Client 0] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=315) Epoch 1: train loss 0.01873687654733658, accuracy 0.7181409295352323, val loss 0.023936026805155986, val accuracy 0.7297297297297297 [repeated 2x across cluster]
(ClientAppActor pid=316) [Client 5] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=316) Epoch 1: train loss 0.018809234723448753, accuracy 0.7209653092006033, val loss 0.024936894031420145, val accuracy 0.684931506849315 [repeated 2x across cluster]
(ClientAppActor pid=316) [Client 10] fit, config: {} [repeated 2x across cluster]
(ClientAppActor pid=316) Epoch 1: train loss 0.01822357065975666, accuracy 0.7390648567119156, val loss 0.02386933483489572, val accuracy 0.7534246575342466 [repeated 2x acr

INFO :      aggregate_fit: received 14 results and 0 failures
INFO :      configure_evaluate: strategy sampled 13 clients (out of 15)


(ClientAppActor pid=315) [Client 5] evaluate, config: {}
(ClientAppActor pid=315) [Client 4] fit, config: {}
(ClientAppActor pid=315) Epoch 1: train loss 0.01775161549448967, accuracy 0.755656108597285, val loss 0.021982648193019712, val accuracy 0.7397260273972602
(ClientAppActor pid=315) [Client 6] evaluate, config: {} [repeated 12x across cluster]


INFO :      aggregate_evaluate: received 13 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 15 rounds in 924.65s
INFO :      History (loss, distributed):
INFO :      	('\tround 1: 44.588891263994434\n'
INFO :      	 '\tround 2: 0.025132255778032804\n'
INFO :      	 '\tround 3: 0.023781624685318217\n'
INFO :      	 '\tround 4: 0.023857067292361218\n'
INFO :      	 '\tround 5: 0.023447425018714746\n'
INFO :      	 '\tround 6: 0.0236784079366288\n'
INFO :      	 '\tround 7: 0.02378794766734001\n'
INFO :      	 '\tround 8: 0.023704675767916916\n'
INFO :      	 '\tround 9: 0.023836830448575364\n'
INFO :      	 '\tround 10: 0.023254359209476057\n'
INFO :      	 '\tround 11: 0.02370471636300357\n'
INFO :      	 '\tround 12: 0.024080303396478056\n'
INFO :      	 '\tround 13: 0.023539918881963516\n'
INFO :      	 '\tround 14: 0.02393073916801904\n'
INFO :      	 '\tround 15: 0.023147609155397316\n')History (metrics, distributed, fit):
INFO :      	{'accuracy':